# Initiate spark

In [1]:
import os

from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession 
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col, max, to_date, expr

from delta.tables import DeltaTable

conf = (
    SparkConf()
    .setAppName("Spark minIO Test")
    .set("spark.hadoop.fs.s3a.endpoint", "http://192.168.86.192:9000")
    .set("spark.hadoop.fs.s3a.access.key", os.getenv('MINIO_ROOT_USER'))
    .set("spark.hadoop.fs.s3a.secret.key", os.getenv('MINIO_ROOT_PASSWORD'))
    .set("spark.hadoop.fs.s3a.path.style.access", True)
    .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .set("spark.driver.memory", "8g")
    .set("spark.executor.memory", "8g")
    .set("spark.delta.logStore.class", "org.apache.spark.sql.delta.storage.S3SingleDriverLogStore") 
    .set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") 
    .set("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") 
)
sc = SparkContext(conf=conf).getOrCreate()
spark = SparkSession(sc).builder.getOrCreate()

21/08/25 21:44:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


### Determine the last load dat to silver

In [13]:
last_load_datetime = spark.read.format('delta').load('s3a://silver-knmi/daggegevens').select(max("load_datetime")).collect()[0]["max(load_datetime)"]
print(last_load_datetime)

2021-08-20 07:55:59.056056


### Read data from the bronze table

In [2]:
df = spark.read.format('delta').load('s3a://bronze-knmi/daggegevens') #.filter(col("load_datetime") > last_load_datetime)
df.select("YYYYMMDD").distinct().show(5)

21/08/25 21:16:29 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


+--------+
|YYYYMMDD|
+--------+
|20081003|
|20010514|
|20150729|
|20160429|
|20160615|
|20080619|
|20040924|
|20110717|
|20100315|
|20200218|
|20180424|
|20020910|
|20040322|
|20180516|
|20030430|
|20021015|
|20130410|
|20020228|
|20100823|
|20210202|
+--------+
only showing top 20 rows



### Transform and rename the columns

In [4]:
from pyspark.sql.functions import trim
silver_df = df.select(trim(col('STN')).alias('weather_station_code'),
                      to_date(col('YYYYMMDD'), 'yyyyMMdd').alias('date'),
                      col('DDVEC').cast("integer").alias("vector_mean_wind_direction_in_degrees"),
                      expr('FHVEC / 10').alias("vector_mean_windspeed_in_meters_per_second"),
                      expr('FG / 10').alias("daily_mean_windspeed_in_meters_per_second"),
                      expr('FHX / 10').alias("max_windspeed_in_meters_per_second"),
                      col('FHXH').alias("max_windspeed_hour_of_day"),
                      expr('FHN / 10').alias("min_windspeed_in_meters_per_second"),
                      expr('FHNH').alias("min_windspeed_hour_of_day"),
                      expr('FXX / 10').alias("max_windgust_in_meters_per_second"),
                      col('FXXH').alias("max_windgust_hour_of_day"),
                      expr('TG / 10').alias("daily_mean_temperature"),
                      expr('TN / 10').alias("minimum_temperature"),
                      col('TNH').alias("minimum_temperature_hour_of_day"),
                      expr('TX / 10').alias("maximum_temperature"),
                      col('TXH').alias("maximum_temperature_hour_of_day"),
                      expr('T10N / 10').alias("max_temp_at_10cm_above_ground"),
                      col('T10NH').alias("max_temp_at_10cm_above_ground_hour_of_day"),
                      expr('SQ / 10').alias("sunshine_duration_in_hours"),
                      expr('SP').alias("percentage_of_max_sumshine_duration"),
                      expr('Q').alias("global_radiation_in_J_per_cm2"),
                      expr('DR / 10').alias("precipitation_duration_in_hours"),
                      expr('RH / 10').alias("daily_precipitaion_amount_in_mm"),
                      expr('RHX / 10').alias("max_hourly_precipitaion_amount_in_mm"),
                      col('RHXH').alias("max_hourly_precipitaion_hour_of_day"),
                      expr('PG / 10').alias("mean_sealevel_pressure_in_hPa"),
                      expr('PX / 10').alias("max_sealevel_pressure_in_hPa"),
                      col('PXH').alias("max_sealevel_pressure_hour_of_day"),
                      expr('PN / 10').alias("min_sealevel_pressure_in_hPa"),
                      col('PNH').alias("min_sealevel_pressure_hour_of_day"),
                      expr('VVN * 100').alias("minimum_visibility_in_m"),
                      col('VVNH').alias("minimum_visibility_hour_of_day"),
                      expr('VVX * 100 + 100').alias("maximum_visibility_in_m"),
                      col('VVXH').alias("maximum_visibility_hour_of_day"),
                      expr('NG').alias("mean_daily_cloud_cover_in_octants"),
                      expr('UG').alias("mean_humidity"),
                      expr('UX').alias("maximim_humidity"),
                      col('UXH').alias("maximum_humidity_hour_of_day"),
                      expr('UN / 10').alias("minimum_humidity"),
                      col('UNH').alias("minimum_humidity_hour_of_day"),
                      expr('EV24 / 10').alias("potential_evapotranspiration_in_mm"),
                      col("load_datetime")
            ).coalesce(16)
silver_df.show(5)

21/08/25 21:24:46 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+----------+-------------------------------------+------------------------------------------+-----------------------------------------+----------------------------------+-------------------------+----------------------------------+-------------------------+---------------------------------+------------------------+----------------------+-------------------+-------------------------------+-------------------+-------------------------------+-----------------------------+-----------------------------------------+--------------------------+-----------------------------------+-----------------------------+-------------------------------+-------------------------------+------------------------------------+-----------------------------------+-----------------------------+----------------------------+---------------------------------+----------------------------+---------------------------------+-----------------------+------------------------------+-----------------------

In [3]:
spark.sql("""
create table if not exists silver_knmi_daggegevens
using delta 
location 's3a://silver-knmi/daggegevens'
""").show()

21/08/25 21:44:46 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


++
||
++
++



In [10]:
spark.sql("""
DESCRIBE silver_knmi_daggegevens
""").show()

+--------------------+---------+-------+
|            col_name|data_type|comment|
+--------------------+---------+-------+
|weather_station_code|   string|       |
|                date|     date|       |
|vector_mean_wind_...|      int|       |
|vector_mean_winds...|   double|       |
|daily_mean_windsp...|   double|       |
|max_windspeed_in_...|   double|       |
|max_windspeed_hou...|   string|       |
|min_windspeed_in_...|   double|       |
|min_windspeed_hou...|   string|       |
|max_windgust_in_m...|   double|       |
|max_windgust_hour...|   string|       |
|daily_mean_temper...|   double|       |
| minimum_temperature|   double|       |
|minimum_temperatu...|   string|       |
| maximum_temperature|   double|       |
|maximum_temperatu...|   string|       |
|max_temp_at_10cm_...|   double|       |
|max_temp_at_10cm_...|   string|       |
|sunshine_duration...|   double|       |
|percentage_of_max...|   string|       |
+--------------------+---------+-------+
only showing top

### Write to the silver table

In [5]:
if not DeltaTable.isDeltaTable(spark, 's3a://silver-knmi/daggegevens'):
    print("Not a delta table, write the full df")
    silver_df.write.format("delta").save('s3a://silver-knmi/daggegevens')
else:
    print("Merging the data")
    silver_knmi_daggegevens_table = DeltaTable.forPath(spark, 's3a://silver-knmi/daggegevens')
    
    silver_knmi_daggegevens_table.alias("current_data") \
      .merge(
        silver_df.alias("new_data"),
        """
        current_data.weather_station_code = new_data.weather_station_code
        and
        current_data.date = new_data.date
        """).whenNotMatchedInsertAll().execute()

Merging the data


KeyboardInterrupt: 

In [7]:
silver_df.createOrReplaceTempView("silver_df")

In [10]:
spark.sql("""
update silver_knmi_daggegevens
set weather_station_code = trim(weather_station_code)
""")

++
||
++
++



In [4]:
spark.sql("""
select distinct weather_station_code, length(weather_station_code)
from silver_knmi_daggegevens
order by  weather_station_code
""").show()

+--------------------+----------------------------+
|weather_station_code|length(weather_station_code)|
+--------------------+----------------------------+
|                 209|                           3|
|                 210|                           3|
|                 215|                           3|
|                 225|                           3|
|                 235|                           3|
|                 240|                           3|
|                 242|                           3|
|                 248|                           3|
|                 249|                           3|
|                 251|                           3|
|                 257|                           3|
|                 258|                           3|
|                 260|                           3|
|                 265|                           3|
|                 267|                           3|
|                 269|                           3|
|           